In [ ]:
"""
knu_score: title과 text에서 knu에 있는 단어를 추출하여 각 단어의 점수를 합한 뒤, 그 단어들의 수 제곱근으로 나눈다. .

*numpy를 사용하여 반복문을 피하고 빠른 계산을 할 수 있다.
*멀티프로세싱으로 시간이 오래 걸리는, 본문으로 단어 리스트 만들기 작업을 하여 총 작업 시간을 줄였다.
"""

import json
import numpy as np

with open('knu/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
    data = json.load(f)

#json파일을 계산에 용이한 numpy array로 변형
data_list = [list(data[i].values()) for i in range(len(data))]
data_array = np.asarray(data_list)

word = np.char.strip(data_array[:,0])
polarity = data_array[:,2]

In [ ]:
def text2array(text):
    """
    제목, 본문에 포함된 단어들을 하나의 array로 변형.
    knu에 기준 단어가 2개 이상의 단어로 이루어진 경우가 많아,
    그런 경우를 위해 2개의 단어로 이루어진 조합을 생성. 
    """
    from eunjeon import Mecab
    import numpy as np
    
    mecab = Mecab()
    lines = text.split(".")
    results = []
    for line in lines:
        wlist = mecab.pos(line.replace('\u2028', '').replace('\x83\x85', ''))
        r = []
        for w in wlist:
            if not w[1] in ['NNBC', 'SF', 'SE', 'SSO', 'SSC', 'SC', 'SY', 'SL', 'SH', 'SN']:
                r.append(w[0])

        rl = (" ".join(r)).strip()
        results.append(rl)

    res = " ".join(results)
    res = res.split()
    comb = ["".join(res[m:n+1])   for m in range(len(res)) for n in range(m,m+2) if n < len(res)]
    comb = np.asarray(comb)
    
    return comb

In [ ]:
import os
import pandas as pd
from pathos.multiprocessing import ProcessingPool as Pool

qlist = ['남자+-남성']#["여성", "여자+-여성", "남성", '남자+-남성']

for query in qlist:
    
    dir_ = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', "#3_knu_score", "#3_"+query)
    if not os.path.exists(dir_):
        os.makedirs(dir_)

    for year in range(2000, 2020):
        
        for month in range(1, 13):
            
            xlsx_name1 = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', "#2_html2text", "#2_merged", '#2_merged_'+query, 
                                     f"{'#2_'+str(year)+'_'+str(month).zfill(2)+'_'+query}.xlsx")

            xlsx_name2 = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_knu_score', "#3_"+query,
                                     f"{'#3_'+str(year)+'_'+str(month).zfill(2)+'_'+query}.xlsx")

            df = pd.read_excel(xlsx_name1, index_col=0)
            SCORE = list()

            for i in range(0, len(df), 12):

                tlist = list()
                for j in range(12):
                    if i + j == len(df):
                        break
                    tlist.append(str(df.loc[i+j, 'title'])+".\n"+str(df.loc[i+j, 'text'])) # title + text

                with Pool(8) as p:
                    clist = p.map(text2array, tlist)

                for comb in clist:
                    true_false = np.in1d(word, comb) # 본문의 단어들과 knu의 단어들 비교
                    t_score = polarity[true_false] # 본문에 포함된 단어들의 점수를 추출
                    if t_score.size:
                        score = sum(t_score.astype(int))/np.sqrt(len(clist)) #len(t_score)
                        SCORE.append(score)
                    else:
                        SCORE.append(0)

            df["knu_score"] = SCORE
            df.to_excel(xlsx_name2)